In [1]:
# Import libraries
import numpy as np
import geopandas as gpd
import momepy
import networkx as nx
# import pandas as pd
# import shapely
# import shapely.geometry as sg
# import matplotlib
# import matplotlib.pyplot as plt
# %matplotlib inline

from lmzintgraf_gp_pref_elicit import dataset, gaussian_process, acquisition_function
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_ccs as utils_ccs
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_data as utils_data
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_experiment as utils_experiment
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_parameters as utils_parameters
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_user as utils_user

In [2]:
map = gpd.read_file("Sidewalk_width_crossings_small.geojson") #Read in the map with radius 250m and ~1000 nodes

# Objectives
objective1 = map['length']
objective2 = map['crossing']
objective3 = map['obstacle_free_width']

objectives = ('length', 'crossing')

In [3]:
# Create a NetworkX graph from the map
G = momepy.gdf_to_nx(map, approach='primal')
nodes = G.nodes
edges = G.edges

/usr/local/anaconda3/envs/test/lib/python3.10/site-packages/momepy/utils.py:252: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_network[length] = gdf_network.geometry.length


In [4]:
# print(nodes)

In [5]:
#Pick random ones or pick manually that make sense - to experiment
S = (120548.6120283842, 486088.19577846595)
T = (120798.0893320718, 486128.7633437495)

In [6]:
# Initialise the Gaussian process for 2 objectives
gp = gaussian_process.GPPairwise(num_objectives=2, std_noise=0.01, kernel_width=0.15,prior_mean_type='zero', seed=None)

In [7]:
P = [] #Pareto set
p = [] #paths computed by Dijkstra's algorithm
val_vector_p = [] #value vectors w.r.t. p, i.e., v^{p_1}, v^{p_2}

# Path initialisation
for i in objectives:
    p = nx.shortest_path(G, source=S, target=T, weight=i, method='dijkstra') #Dijkstra's algorithm
    P.append(p)

    val_obj1 = nx.path_weight(G, path=p, weight='length') #Returns total cost associated with the path and weight. In other words, it returns the value of the path.
    val_obj2 = nx.path_weight(G, path=p, weight='crossing')
    val_vector_p.append(np.array([val_obj1, val_obj2]))

In [8]:
val_vector_p

[array([671.64,   3.  ]), array([674.81,   3.  ])]

In [9]:
C = [min(val_vector_p[0][0], val_vector_p[1][0]), min(val_vector_p[0][1], val_vector_p[1][1])] #Candidate Targets, i.e., the most optimistic points
C

[671.6400000000003, 3.0]

In [10]:
# User ranking: Compare paths in P
user_preference = utils_user.UserPreference(num_objectives=2, std_noise=0.1)

In [11]:
add_noise = True
ground_utility = user_preference.get_preference(val_vector_p, add_noise=add_noise) #This is the ground-truth utility, i.e., the true utility
print(ground_utility)

[1.13993612 1.13491129]


/Users/alisatodorova/Desktop/Thesis/Interactive-MOPPA/lmzintgraf_gp_pref_elicit/gp_utilities/utils_user.py:254: RuntimeWarning: overflow encountered in exp
  y += 1. / (1 + np.exp(- x * (a-i) + (b+i)))


In [12]:
# Add the comparisons to GP
comparisons = dataset.DatasetPairwise(num_objectives=2)

comparisons.add_single_comparison(val_vector_p[np.argmax(ground_utility)], val_vector_p[np.argmin(ground_utility)]) #This way we are performing user ranking of their preferences
print(comparisons.datapoints)
gp.update(comparisons)

[[671.64   3.  ]
 [674.81   3.  ]]


In [13]:
# Find the path the user likes best and has the maximum a posteriori (MAP) estimate
u_v, _ = gp.get_predictive_params(val_vector_p, True) #The maximum a posteriori (MAP) estimate is the mean from gaussian_process.get_predictive_params()
print(u_v)

[ 0.02560529 -0.02560529]


In [14]:
p_star_index = np.argmax(u_v)
p_star_index

0

In [15]:
p_star = P[p_star_index]
p_star

[(120548.6120283842, 486088.19577846595),
 (120558.58272730978, 486088.5913559315),
 (120562.22249291666, 486088.4604878346),
 (120597.87366172913, 486118.3885286597),
 (120603.94807229919, 486112.8244804511),
 (120610.1211584085, 486110.4734641692),
 (120615.236697444, 486109.4981894336),
 (120622.18953041689, 486108.8516084536),
 (120627.6055477486, 486108.80491815176),
 (120637.3711853829, 486107.4803808584),
 (120638.32762784461, 486107.6578267999),
 (120644.82334925648, 486107.55756407854),
 (120646.58367929762, 486107.4539825946),
 (120653.63177241654, 486106.9806947992),
 (120658.16234661153, 486106.63926980837),
 (120664.93105615664, 486106.10542439454),
 (120671.8192466126, 486105.5397502568),
 (120678.46251254133, 486104.92881432245),
 (120683.05468505199, 486104.3662392544),
 (120689.70658339545, 486103.4113373837),
 (120697.63022968677, 486102.02209520293),
 (120704.74846380341, 486100.38072378177),
 (120714.24315622302, 486097.30355140124),
 (120715.2451666701, 486096.9044

In [16]:
# #TODO: No idea if this is correct.
# input_domain = utils_ccs.get_pcs_grid(ccs_size=10, num_objectives=2, eucledian_dist=0.005, min_size=10, seed=123)
# input_domain = np.array(input_domain)
# #Set eucledian_dist smaller to run the code faster
# # Setting ccs_size and min_size to the same value is probably also a good idea.
# # print(input_domain)

In [17]:
input_domain = np.array([C])
print(input_domain)

[[671.64   3.  ]]


In [18]:
# Initialise the acquisition function
#set of candidate targets
acq_fun = acquisition_function.DiscreteAcquirer(input_domain=input_domain, query_type='ranking', seed=123, acquisition_type='expected improvement')

In [19]:
#TODO: The next code cells are in a while-loop
# while C:
#     expected_improvement = acquisition_function.get_expected_improvement(input_domain, gp, acq_fun.history)
#     t_index = np.argmax(expected_improvement)
#     t = C[t_index]
#     C.remove(t)
#
# # t_index
# print(t)

In [20]:
expected_improvement = acquisition_function.get_expected_improvement(input_domain, gp, acq_fun.history)
print(expected_improvement)

[0.14549099]


In [21]:
t_index = np.argmax(expected_improvement)
t_index

0

In [22]:
t = C[t_index]
t

671.6400000000003

In [23]:
# Remove t from C
C.remove(t)
print(C)

[3.0]


In [24]:
# Inner-loop approach
import dfs_lower
p_s = dfs_lower(G, S, t, lower_bounds) #TODO: I think I have forgotten to add the target in dfs_lower :'(

NameError: name 'lower_bounds' is not defined

In [ ]:
#TODO: Line 15 of pseudocode is unclear to me how it should be in code. Also, which is correct:
# If v^p improves in the target region
# because you've identified a new value vector on the PCS. If you stop once the utility no longer improves, I think this can result in stopping prematurely. Specifically, imagine you have a current partial Pareto front of (10,0) and (0,10) the user model u((10,0)) is the current best. The target vector is (10,10)  and when you run DFS, you get one of the possible vectors in the target region. You get (1,9) out of the the call to DFS, and the u((1,9)) < u((10,0)) even after querying the user about it. Now here, you shouldn't stop, because the true best - (7,3) for example, is still possible to find.
#you're just not going to improve on that with a newly found vector
#So improving on the acquisition function by identifying a new point is impossible as
# 1) you were searching at an optimistic estimate (target), so the actually found value will be worse than the target
# 2) finding new points, and querying the user reduces uncertainty

In [ ]:
P = P.append(p_s)

In [35]:
#TODO: Check if this is correct bc for the sample example the input looks different so i can't check
v_n = {}
threshold=1e-8
max_iter=1000
for i in range(max_iter): #or until convergence
    v_n_copy = v_n.copy()
    for n in G:
        max_value = -np.inf
        for n_next in G[n]:
            for key in G:
                cost = key
                result = np.min(cost + n_next)
                max_value = max(max_value, result)
            v_n[n] = max_value

        converged = all(n in v_n_copy and abs(v_n[n] - v_n_copy[n]) < threshold for n in v_n)
        if converged:
            break

print(v_n)

# import single_vi_iter
# lower_bounds = single_vi_iter.single_value_iter(G)
# lower_bounds

{(120548.6120283842, 486088.19577846595): 120558.58272730978, (120558.58272730978, 486088.5913559315): 120562.22249291666, (120562.22249291666, 486088.4604878346): 120597.87366172913, (120554.77791147187, 486105.081631085): 120558.22483637692, (120558.22483637692, 486103.0489683649): 120561.12010043039, (120561.12010043039, 486102.03679359506): 120577.53919356126, (120549.11715177551, 486040.41438763676): 120558.13567293479, (120558.13567293479, 486044.5374951657): 120567.28178312563, (120567.28178312563, 486048.45472129877): 120570.33350867042, (120570.33350867042, 486049.6733418503): 120579.49075192933, (120565.93040988805, 485974.75610421225): 120575.18243990708, (120556.76794233406, 485970.8238889219): 120565.93040988805, (120555.18193758334, 485970.13570051384): 120556.76794233406, (120575.18243990708, 485978.41728733847): 120576.81484759072, (120576.81484759072, 485978.27265655366): 120580.87871553846, (120579.49075192933, 486053.578854089): 120588.64528634987, (120588.6452863498

In [ ]:
#TODO: More user ranking...
#Compare p^s to p^∗ and add comparison to the GP ▷User ranking, i.e., is the new path preferred to the current, maximum one?

val_vector_paths = [] #value vectors of paths p^s and p^∗

val_p_s = nx.path_weight(G, path=p_s, weight='length') #TODO: weight is what?
val_p_star = nx.path_weight(G, path=p_star, weight='crossing')

val_vector_paths.append(np.array([val_p_s, val_p_star]))


In [ ]:
ranking_paths = user_preference.get_preference(val_vector_paths, add_noise=add_noise)
print(ranking_paths)

In [ ]:
# Add the comparisons to GP
comparisons.add_single_comparison(val_vector_paths[np.argmax(ranking_paths)], val_vector_paths[
    np.argmin(ranking_paths)])  #This way we are performing user ranking of their preferences
print(comparisons.datapoints)
gp.update(comparisons)

In [ ]:
# if u(v^{p^s}) > u(v^{p^*}) then
# p^∗ ← p^s
# end if


In [ ]:
# Compute new candidate targets based on v^{p^s} and add to C


In [ ]:
# end if
# end while
# return p^∗, v^{p^*}
